# 데이터 분석을 위한 추가 전처리
1. 'stos', 'dtos' 제거
2. 'label' 에서 'target 생성 및 label 제거
3. 숫자형 데이터 MinMaxScaling

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset

from scipy.optimize import linear_sum_assignment as linear_assignment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


In [48]:
dataset=pd.read_csv('/content/drive/MyDrive/캡스톤/dataset/CTU-13(13)_preprocessed.csv')

In [49]:
# column 명들 소문자로 변환
dataset.columns = dataset.columns.str.lower()

In [50]:
# 'stos0' 'dtos0' 모두 1.0 의 값을 가지므로 분류에 필요하지 않은 것 같아 삭제 (10번 시나리오에서는)
dataset.drop(['stos0','dtos0'],axis=1,inplace=True)
dataset

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport3,dport4,state0,state1,state2,state3,totpkts,totbytes,srcbytes,label
0,2460.003906,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,6,412,272,flow=From-Normal-V54-Stribrek
1,2458.846924,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,8,488,248,flow=From-Botnet-V54-TCP-Attempt
2,2452.237549,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,8,488,248,flow=From-Botnet-V54-TCP-Attempt
3,2451.636475,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,8,488,248,flow=From-Botnet-V54-TCP-Attempt
4,2467.759033,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,8,488,248,flow=From-Botnet-V54-TCP-Attempt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71937,0.000429,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,2,400,74,flow=From-Normal-V54-Stribrek
71938,0.018334,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,6,412,272,flow=From-Normal-V54-Stribrek
71939,0.000521,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,2,400,74,flow=From-Normal-V54-Stribrek
71940,0.000242,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,2,400,74,flow=From-Normal-V54-Stribrek


In [51]:
# Botnet 과 Normal 구분 -> 정상 0, 비정상 1
def convertLabel(sample):
  if 'Botnet' in sample: return 1
  else: return 0

dataset['target'] = dataset['label'].apply(convertLabel)

In [52]:
# label feature 제거
dataset.drop(['label'],axis=1,inplace=True)

In [53]:
# 숫자형 feature -> MinMaxScaling
columns=['dur','totpkts','totbytes','srcbytes']
scaler=MinMaxScaler()
dataset[columns]=scaler.fit_transform(dataset[columns])

## Train data, Test data 나누기
**논문에 나와있는 대로**
1. Train data -> Normal data(40%)
2. Test data -> Normal data(60%) + Abnormal data(60%)

In [54]:
# 정상 dataset / 비정상 dataset
normal_dataset=dataset[dataset['target']==0]
abnormal_dataset=dataset[dataset['target']==1]

# 정상 dataset -> Train 정상 / Test 정상 나누기 , 40%:60%
normal_dataset = normal_dataset.drop(['target'],axis=1) # 'target' feature drop 
train_normal,test_normal=train_test_split(normal_dataset, test_size=0.6, random_state=42)

In [55]:
train_normal

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport2,dport3,dport4,state0,state1,state2,state3,totpkts,totbytes,srcbytes
25936,6.852973e-05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000139,0.000014,6.533849e-06
57196,6.723778e-05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000139,0.000014,6.533849e-06
70875,5.101242e-04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.000324,0.000042,1.684043e-05
69759,2.731389e-06,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000002,8.926023e-07
66141,8.666667e-08,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000004,8.807010e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68715,8.722223e-08,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000002,1.071123e-06
19096,1.222222e-07,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000004,8.807010e-07
2732,7.027778e-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000002,9.640105e-07
42534,1.277778e-07,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000003,9.640105e-07


In [56]:
test_normal

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport2,dport3,dport4,state0,state1,state2,state3,totpkts,totbytes,srcbytes
48682,1.084411e-04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000216,0.000023,1.041369e-05
29373,1.211111e-07,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000003,9.640105e-07
25897,5.061945e-06,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000093,0.000005,3.237171e-06
64147,6.416667e-08,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000002,8.807010e-07
42805,1.763889e-07,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000004,8.807010e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38482,1.141667e-07,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000004,8.807010e-07
21712,4.920000e-06,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000077,0.000004,3.237171e-06
69012,7.333334e-08,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000002,9.283064e-07
20548,8.888889e-08,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000004,8.807010e-07


In [57]:
# 비정상 dataset에서 random 으로 60% 추출
test_abnormal = abnormal_dataset.sample(frac=0.6)
test_abnormal = test_abnormal.drop(['target'],axis=1) # 'target' feature drop 

In [58]:
test_abnormal

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport2,dport3,dport4,state0,state1,state2,state3,totpkts,totbytes,srcbytes
9119,9.972223e-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000005,8.449969e-07
9978,2.975599e-02,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.003288,0.002007,1.943933e-03
53115,3.629975e-04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000077,0.000004,2.213654e-06
19259,3.744934e-04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000077,0.000004,2.213654e-06
10783,2.503463e-03,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000031,0.000001,2.213654e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34742,3.633050e-04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000077,0.000004,2.213654e-06
69394,3.632734e-04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000077,0.000004,2.213654e-06
58846,3.632167e-04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000077,0.000004,2.213654e-06
55455,3.632567e-04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000077,0.000004,2.213654e-06


# Dataframe 을 pytorch tensor로 변환 

In [59]:
# pytorch tensor 로 변환 

# determine the supported device
def get_device():
  if torch.cuda.is_available():
    device = torch.device('cuda:0')
  else :
    device = torch.device('cpu') # don 't have GPU 
  return device

# convert a df to tensor to be used in pytorch
def dataframe_to_tensor(df):
   device = get_device()
   return torch.from_numpy(df.values).float().to(device)

In [60]:
train_normal = dataframe_to_tensor(train_normal)

# Deep K-means
1. layer 수는 5
2. mini-batch size 는 100 
3. 각 layer에서 얼만큼 줄일지 + 활성화 함수는 어떤 것을 사용할지 ?
4. learning rate=0.001 인데 어느 정도로 해야할지?

--> 3차원으로 latent vector 크기 잡아도 될듯

In [61]:
# train_loader 생성
train_loader = DataLoader(train_normal,batch_size=100,shuffle=True)

In [62]:
class TestDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len


In [63]:
# test_loader 생성
test_data = pd.concat([test_normal, test_abnormal]).values
test_label = np.concatenate([np.zeros(19164),np.ones(24002)])

test_data = TestDataset(test_data,test_label)

test_loader = DataLoader(test_data,batch_size=100,shuffle=True)

In [64]:
class Encoder(nn.Module):
  def __init__(self,latent_size):
    super(Encoder,self).__init__()

    self.encoder = nn.Sequential(
                      nn.Linear(27,16),
                      nn.ReLU(),
                      nn.Linear(16,latent_size),
                      nn.ReLU())
          
    
  def forward(self, x):
    return self.encoder(x)

class Decoder(nn.Module):
  def __init__(self, latent_size):
    super(Decoder,self).__init__()

    self.decoder = nn.Sequential(
                      nn.Linear(latent_size,16),
                      nn.ReLU(),
                      nn.Linear(16,27),
                      nn.Sigmoid())
    
  def forward(self,x):
    return self.decoder(x)

In [65]:
class Kmeans(nn.Module):
  def __init__(self,num_clusters, latent_size):
    super(Kmeans,self).__init__()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.num_clusters = num_clusters
    self.centroids = nn.Parameter(torch.rand((self.num_clusters,latent_size)).to(device))

  def argminl2distance(self,a,b):
    return torch.argmin(torch.sum((a-b)**2,dim=1),dim=0)

  def forward(self,x):
    y_assign = []
    for m in range(x.size(0)):
      h = x[m].expand(self.num_clusters,-1)
      assign = self.argminl2distance(h, self.centroids)
      y_assign.append(assign.item())
    return y_assign, self.centroids[y_assign]

In [66]:
def cluster_acc(y_true,y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  D = max(y_pred.max(), y_true.max()) + 1
  w = np.zeros((D,D), dtype = np.int64)
  for i in range(y_pred.size):
    w[y_pred[i],y_true[i]] +=1
  ind = linear_assignment(w.max() - w)
  return sum([w[i,j] for i,j in zip(ind[0],ind[1])]) * 1.0 / y_pred.size

In [67]:
def evaluation(testloader, encoder, kmeans, device):
  predictions = []
  actual = []

  with torch.no_grad():
    for images, labels in testloader:
      inputs = images.to(device)
      labels = labels.to(device)
      latent_var = encoder(inputs)
      y_pred,_ = kmeans(latent_var)

      predictions+=y_pred
      actual+=labels.cpu().tolist()

  return cluster_acc(actual,predictions)

#모델 학습


In [164]:
latent_size=2
num_clusters=2

In [165]:
encoder = Encoder(latent_size).to(device)
decoder = Decoder(latent_size).to(device)
kmeans = Kmeans(num_clusters, latent_size).to(device)
criterion1 = torch.nn.MSELoss()
criterion2 = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters())+list(kmeans.parameters()), lr=0.001)

In [166]:
T1=100
T2=200
lam = 1e-3

In [167]:
for ep in range(300):
  if ep <= T1:
    alpha = lam/(T2-T1)
  else:
    alpha = lam
  
  # 정상 data(label=0)에 대해서만 학습
  running_loss = 0.0
  for batch in train_loader:
    inputs = batch.to(device)
    optimizer.zero_grad()
    latent_var = encoder(inputs)
    _, centroids = kmeans(latent_var.detach())
    outputs = decoder(latent_var)

    l_rec = criterion1(inputs,outputs)
    l_clt = criterion2(latent_var,centroids)
    loss = l_rec + alpha*l_clt

    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  
  ###
  avg_loss = running_loss/len(train_loader)

  if(ep%10==0):
    print('[%d] Train loss:%.4f' %(ep,avg_loss) )

[0] Train loss:0.1769
[10] Train loss:0.0027
[20] Train loss:0.0018
[30] Train loss:0.0015
[40] Train loss:0.0013
[50] Train loss:0.0011
[60] Train loss:0.0010
[70] Train loss:0.0010
[80] Train loss:0.0010
[90] Train loss:0.0010
[100] Train loss:0.0009
[110] Train loss:0.0010
[120] Train loss:0.0010
[130] Train loss:0.0010
[140] Train loss:0.0009
[150] Train loss:0.0009
[160] Train loss:0.0008
[170] Train loss:0.0008
[180] Train loss:0.0008
[190] Train loss:0.0008
[200] Train loss:0.0008
[210] Train loss:0.0008
[220] Train loss:0.0008
[230] Train loss:0.0008
[240] Train loss:0.0008
[250] Train loss:0.0008
[260] Train loss:0.0008
[270] Train loss:0.0008
[280] Train loss:0.0008
[290] Train loss:0.0008


In [168]:
torch.save(encoder.state_dict(),'/content/drive/MyDrive/캡스톤/models(CTU-13(13))/encoder.pth')
torch.save(decoder.state_dict(),'/content/drive/MyDrive/캡스톤/models(CTU-13(13))/decoder.pth')
torch.save(kmeans.state_dict(),'/content/drive/MyDrive/캡스톤/models(CTU-13(13))/kmeans.pth') 

# 학습 후 One class svm ensemble

In [242]:
encoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(CTU-13(13))/encoder.pth'))
decoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(CTU-13(13))/decoder.pth'))
kmeans.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(CTU-13(13))/kmeans.pth'))

<All keys matched successfully>

In [243]:
# svm 에 활용할 latent_vectors 생성
# svm 에 활용할 latent_vectors_labels 생성

train_latent_vectors = [] # latent vector 가 list로 계속해서 들어간 형태
train_predict_label= []

for data in train_loader:
  inputs = data.to(device)
  latent_vector= encoder(inputs)
  predict_label, _ = kmeans(latent_vector)

  train_latent_vectors+=latent_vector.cpu().tolist()
  train_predict_label+=predict_label


In [244]:
# svm 에 활용할 test_latent_vectors 생성
# svm 에 활용할 test_label (실제 label 값) 생성

test_latent_vectors = [] # latent vector 가 list로 계속해서 들어간 형태
test_predict_label = []
test_label= []

for data,labels in test_loader:
  inputs = data.to(device)
  latent_vector= encoder(inputs)
  predict_label, _ = kmeans(latent_vector)

  test_latent_vectors+=latent_vector.cpu().tolist()
  test_predict_label+=predict_label
  test_label+=labels.cpu().tolist()


# OneSvm Ensemble

In [245]:
#OneclassSVM Ensemble
from sklearn.svm import OneClassSVM
import numpy as np
import pandas as pd

class OCSVMEnsemble():

  def __init__(self,nu=0.5):
    self.nu = nu
    #self.gamma = gamma

  def fit(self, latent, pred):
    # oneclasssvm 인스턴스가 들어갈 리스트
    self.instance = []
    self.num_cluster = len(np.unique(pred))

    # 경계가 9개 생긴거
    for clu in range(self.num_cluster):
      idx = np.where(pred == clu) # 0-9 까지의 각 그룹에 해당하는 index 도출 
      #해당 군집에 속한 data
      clu_data = latent[idx]
      ocsvm = OneClassSVM(kernel = 'rbf',gamma = 'scale', nu = self.nu).fit(clu_data) # OneClassSVM 을 통한 객체 ocsvm 생성
      self.instance.append(ocsvm)

  # 모든 OneClassSVM 인스턴스를 돌면서
  # inlier -> 0 , outlier -> 1 return 하는 함수
  def predictLabel(self,x):
    for model in self.instance:
      model_predict = model.predict(x) # inliers 1, outliers -1
      if model_predict == 1:
        return 0
    return 1
  
  # test 데이터가 할당된 군집에 대한 svm 만을 이용해서 이상 데이터를 분류한다.
  def l_predict(self,x,cluster_label):
    model = self.instance[cluster_label]
    model_predict = model.predict(x)
    if model_predict == 1:
      return 0
    else:
      return 1

In [247]:
# train_latent_vectors, latent_vectors_labels numpy 로 변환
train_latent_vectors=np.array(train_latent_vectors)
train_predict_label=np.array(train_predict_label)

In [248]:
# test_latent_vectors, test_label numpy 로 변환
test_latent_vectors = np.array(test_latent_vectors)
test_label = np.array(test_label)
test_predict_label = np.array(test_predict_label)

In [249]:
# 학습 과정
ocsvm = OCSVMEnsemble(nu=0.36) 
ocsvm.fit(train_latent_vectors, train_predict_label) # train data 에서 latent vector 들 + 가장 가까운 중심의 index

In [177]:
kmeans.centroids

Parameter containing:
tensor([[1.8682, 0.0688],
        [3.2641, 8.6733]], device='cuda:0', requires_grad=True)

In [178]:
train_latent_vectors[0:20]

array([[1.80007446, 0.        ],
       [1.80003214, 0.        ],
       [3.47599959, 8.62066174],
       [1.80006683, 0.        ],
       [3.47303581, 8.6096859 ],
       [3.47601414, 8.6206646 ],
       [1.86386716, 0.17960973],
       [3.11171126, 8.72570896],
       [3.11105156, 8.72458649],
       [0.        , 0.        ],
       [1.86386716, 0.17960973],
       [1.80007493, 0.        ],
       [1.86386716, 0.17960973],
       [1.80007446, 0.        ],
       [1.86386716, 0.17960973],
       [1.80007529, 0.        ],
       [1.86386716, 0.17960973],
       [3.47599936, 8.62066174],
       [1.86386669, 0.17960927],
       [1.86386716, 0.17960973]])

In [79]:
train_predict_label[0:20]

array([0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1])

In [102]:
test_latent_vectors[10:20]

array([[2.94776154, 7.35063362, 3.32169414],
       [3.23242879, 2.37119436, 3.82463384],
       [3.23011661, 2.37015009, 3.82201529],
       [3.22983527, 2.37009335, 3.82193208],
       [2.94775271, 7.35062933, 3.32168722],
       [3.23241472, 2.37118769, 3.82461762],
       [2.94776344, 7.35063553, 3.3216958 ],
       [2.53637481, 6.49470711, 2.97293186],
       [3.22972894, 2.36997223, 3.82156992],
       [2.94775271, 7.35062933, 3.32168722]])

In [103]:
test_label[10:20]

array([1, 1, 1, 0, 0, 1, 0, 0, 0, 0])

In [240]:
# test과정에서의 latent vector 들을 넣어가면서 예측하는 과정
y_pred = []
for i in test_latent_vectors:
  i=i.reshape(1,-1)
  pred=ocsvm.predictLabel(i)
  y_pred.append(pred)

In [241]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(test_label,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.57      0.71     19164
           1       0.74      0.97      0.84     24002

    accuracy                           0.79     43166
   macro avg       0.84      0.77      0.77     43166
weighted avg       0.83      0.79      0.78     43166



In [250]:
y_pred2 = []
for i,j in zip(test_latent_vectors,test_predict_label):
  i=i.reshape(1,-1)
  pred=ocsvm.l_predict(i,j)
  y_pred2.append(pred)

In [251]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(test_label,y_pred2))

              precision    recall  f1-score   support

           0       0.94      0.56      0.70     19164
           1       0.73      0.97      0.83     24002

    accuracy                           0.79     43166
   macro avg       0.83      0.76      0.77     43166
weighted avg       0.82      0.79      0.77     43166

